In [46]:
import math  # Just ignore this :-)

def log(x):
    if x == 0:
        return float('-inf')
    return math.log(x)

# CTiB E2023 - Week 11 - Exercises

# Theoretical exercises

***Exercise 1***: How many terms are there in the sum on slide 37 from the lecture on Nov 13 for computing $P({\bf X}|\Theta)$? Why?



*This equation is the probability of the observable sequence given a set of parameters $\Theta$. To calculate this we need to find the sum of probabilities of all possible hiddens paths. For each of the hidden paths, Z, there is $2\cdot N$ terms, since the first position need the initial probability and emission probability and all the rest needs transition probability and emission probability. If there are k hidden states, then there is $k^N$ possible hidden state paths. This gives us in total $2\cdot N\cdot k^N$ terms.*

***Exercise 2***: How many terms are there in the maximization on slide 4 from the lecure on Nov 20 for computing the Viterbi decoding ${\bf Z}^*$? Why?



*To find the maximal path probability for a hidden state path you need to fill out the w-table. To do this you first fill the first column. This is done by taking the initial probability times the emission probability. You do this for K positions, so the first column takes $2 \cdot K$ terms. For all, $N-1$, subsequent columns and for each of the $K$ position you calculate $K$ probabilites and fills in the max. This is means finding Z* takes $2\cdot K+(N-1)\cdot(K\cdot(K\cdot 2+1))$ terms.*

***Exercise 3***: Where in the derivation of $\omega({\bf z}_n$) on slide 8 do we use that the fact that we are working with hidden Markov models? And how do we use it?

*Already in the second step where we calculate the probability of the next position in the sequence only based on the state of the previous position.*

***Exercise 4***: How many terms are there in the maximixation on slide 2 in the Posterior decoding slides from Nov 20 for computing ${\bf z}^*_n$, i.e. the *n*th state in a posterior decoding? Why?



*Even though the forward and the backward algorithm works differently they both need the sum of $K \cdot 2$ terms to find the probability of a given position. Since there are K positions in each column and N columns; there are $N \cdot K \cdot (2 \cdot K + 1)$ terms to find the $z_n^*$. <span style="color:red">er ikke helt rigtig i think!</span>*

Skal rettes til K

***Exercise 5***: Why is $P({\bf X}) = \sum_{{\bf z}_n} \alpha({\bf z}_n) \beta({\bf z}_n) = \sum_{{\bf z}_N} \alpha({\bf z}_N)$ as stated on slide 5 in the Posterior decoding slides?



*Since each column of the table is the sum of probabilities of all paths leading to that position; the sum of each column should be close to 1. If it is not close to 1 that indicates that the model is not very good because the the observed sequence, X, is unlikely.<span style="color:red">lidt mangelfuldt måske.</span>*

Skal rettes 

***Exercise 6***: Algorithmic question: Slide 20 in the Posterior decoding slides shows how to compute $P({\bf X})$ from $\alpha({\bf z}_N)$ in time $O(K^2N)$, i.e. the time it takes to compute the last (rightmost) colummn in the $\alpha$-table. How much space do you need to compute this column? Do you need to store the entire $\alpha$-table?

*Because you can always calculate the next column based on the previous column you only have to save the last column. You therefore do not need to save the entire $\alpha$-table; You can instead save the one column at a time to calculate the values for the next column. This means you can limit memory usage to two columns of $K$-length or $2\cdot K$.<span style="color:red">tænker dette er rigtigt</span>*

# Practical exercises

You are given the same 7-state HMM and helper functions that you used last week:

In [47]:
class hmm:
    def __init__(self, init_probs, trans_probs, emission_probs):
        self.init_probs = init_probs
        self.trans_probs = trans_probs
        self.emission_probs = emission_probs

In [48]:
init_probs_7_state = [0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00]

trans_probs_7_state = [
    [0.00, 0.00, 0.90, 0.10, 0.00, 0.00, 0.00],
    [1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.05, 0.90, 0.05, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00],
    [0.00, 0.00, 0.00, 0.10, 0.90, 0.00, 0.00],
]

emission_probs_7_state = [
    #   A     C     G     T
    [0.30, 0.25, 0.25, 0.20],
    [0.20, 0.35, 0.15, 0.30],
    [0.40, 0.15, 0.20, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.20, 0.40, 0.30, 0.10],
    [0.30, 0.20, 0.30, 0.20],
    [0.15, 0.30, 0.20, 0.35],
]

hmm_7_state = hmm(init_probs_7_state, trans_probs_7_state, emission_probs_7_state)

In [49]:
def translate_observations_to_indices(obs):
    mapping = {'a': 0, 'c': 1, 'g': 2, 't': 3}
    return [mapping[symbol.lower()] for symbol in obs]

def translate_indices_to_observations(indices):
    mapping = ['a', 'c', 'g', 't']
    return ''.join(mapping[idx] for idx in indices)

def translate_path_to_indices(path):
    return list(map(lambda x: int(x), path))

def translate_indices_to_path(indices):
    return ''.join([str(i) for i in indices])

# 1 - Viterbi Decoding

Below you will implement and experiment with the Viterbi algorithm. The implementation has been split into three parts:

1. Fill out the $\omega$ table using the recursion presented at the lecture.
2. Find the state with the highest probability after observing the entire sequence of observations.
3. Backtrack from the state found in the previous step to obtain the optimal path.

We'll be working with the 7-state model (`hmm_7_state`) and the helper function for translating between observations, hidden states, and indicies, as introduced above (and also used last week).

Additionally, you're given the function below that constructs a table of a specific size filled with zeros.

In [50]:
def make_table(m, n):
    """Make a table with `m` rows and `n` columns filled with zeros."""
    return [[0] * n for _ in range(m)]

You'll be testing your code with the same two sequences as last week, i.e:

In [51]:
x_short = 'GTTTCCCAGTGTATATCGAGGGATACTACGTGCATAGTAACATCGGCCAA'
z_short = '33333333333321021021021021021021021021021021021021'

In [52]:
x_long = 'TGAGTATCACTTAGGTCTATGTCTAGTCGTCTTTCGTAATGTTTGGTCTTGTCACCAGTTATCCTATGGCGCTCCGAGTCTGGTTCTCGAAATAAGCATCCCCGCCCAAGTCATGCACCCGTTTGTGTTCTTCGCCGACTTGAGCGACTTAATGAGGATGCCACTCGTCACCATCTTGAACATGCCACCAACGAGGTTGCCGCCGTCCATTATAACTACAACCTAGACAATTTTCGCTTTAGGTCCATTCACTAGGCCGAAATCCGCTGGAGTAAGCACAAAGCTCGTATAGGCAAAACCGACTCCATGAGTCTGCCTCCCGACCATTCCCATCAAAATACGCTATCAATACTAAAAAAATGACGGTTCAGCCTCACCCGGATGCTCGAGACAGCACACGGACATGATAGCGAACGTGACCAGTGTAGTGGCCCAGGGGAACCGCCGCGCCATTTTGTTCATGGCCCCGCTGCCGAATATTTCGATCCCAGCTAGAGTAATGACCTGTAGCTTAAACCCACTTTTGGCCCAAACTAGAGCAACAATCGGAATGGCTGAAGTGAATGCCGGCATGCCCTCAGCTCTAAGCGCCTCGATCGCAGTAATGACCGTCTTAACATTAGCTCTCAACGCTATGCAGTGGCTTTGGTGTCGCTTACTACCAGTTCCGAACGTCTCGGGGGTCTTGATGCAGCGCACCACGATGCCAAGCCACGCTGAATCGGGCAGCCAGCAGGATCGTTACAGTCGAGCCCACGGCAATGCGAGCCGTCACGTTGCCGAATATGCACTGCGGGACTACGGACGCAGGGCCGCCAACCATCTGGTTGACGATAGCCAAACACGGTCCAGAGGTGCCCCATCTCGGTTATTTGGATCGTAATTTTTGTGAAGAACACTGCAAACGCAAGTGGCTTTCCAGACTTTACGACTATGTGCCATCATTTAAGGCTACGACCCGGCTTTTAAGACCCCCACCACTAAATAGAGGTACATCTGA'
z_long = '3333321021021021021021021021021021021021021021021021021021021021021021033333333334564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564563210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210321021021021021021021021033334564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564563333333456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456332102102102102102102102102102102102102102102102102102102102102102102102102102102102102102102102103210210210210210210210210210210210210210210210210210210210210210'

Remember to translate these sequences to indices before using them with your algorithms.

## Implementing without log-transformation

First, we will implement the algorithm without log-transformation. This will cause issues with numerical stability (like above when computing the joint probability), so we will use the log-transformation trick to fix this in the next section.

### Computation of the $\omega$ table

In [53]:
def compute_w(model, x):
    k = len(model.init_probs)
    n = len(x)
    x = translate_observations_to_indices(x)
    
    w = make_table(k, n)
    
    # Base case: fill out w[i][0] for i = 0..k-1
    for j in range(k):
        ip = model.init_probs[j]
        ep = model.emission_probs[j][x[0]]
        w[j][0] = ip * ep
    
    # Inductive case: fill out w[i][j] for i = 0..k, j = 0..n-1
    for i in range(1, n):
        for j in range(k):
            max_prob = 0
            for l in range(k):
                tp = model.trans_probs[l][j]
                ep = model.emission_probs[j][x[i]]
                prob = w[l][i-1] * tp * ep
                if prob > max_prob:
                    max_prob = prob
            w[j][i] = max_prob
    return w          


### Finding the joint probability of an optimal path

Now, write a function that given the $\omega$-table, returns the probability of an optimal path through the HMM. As explained in the lecture, this corresponds to finding the highest probability in the last column of the table.

In [54]:
def opt_path_prob(w):
    max_prob = 0
    for j in range(len(w)):
        prob = w[j][-1]
        if prob > max_prob:
            max_prob = prob
    return max_prob


Now test your implementation in the box below:

In [55]:
w = compute_w(hmm_7_state, x_short)
opt_path_prob(w)

1.9114255184318858e-31

Now do the same for `x_long`. What happens?

In [56]:
w = compute_w(hmm_7_state, x_long)
opt_path_prob(w)

0

### Obtaining an optimal path through backtracking

Implement backtracking to find a most probable path of hidden states given the $\omega$-table.

In [57]:
def backtrack(model, x, w):
    x = translate_observations_to_indices(x)
    
    max_prob = 0
    max_prob_position = None 
    for j in range(len(w)):
        prob = w[j][-1]
        if prob > max_prob:
            max_prob = prob
            max_prob_position = j
    path = str(max_prob_position)

    pre_prob = max_prob
    pre_prob_position = max_prob_position
    
    for i in range(-2, -len(x)-1, -1):
        for j in range(len(w)):
            tp = model.trans_probs[j][pre_prob_position]
            ep = model.emission_probs[pre_prob_position][x[i+1]]
            prob = w[j][i] * tp * ep
            if math.isclose(prob, pre_prob):
                pre_prob = w[j][i]
                pre_prob_position = j
                path = str(j) + path

    return path

In [58]:
w = compute_w(hmm_7_state, x_short)
z_viterbi = backtrack(hmm_7_state, x_short, w)
print(z_viterbi)

33333333333321021021021021021021021021021021021021


Now do the same for `x_long`. What happens?

In [59]:
# w = compute_w(hmm_7_state, x_long)
# z_viterbi = backtrack(hmm_7_state, x_long, w)
# print(z_viterbi)

*It doesn't work because most of the probabilities are reduced to 0.*

## Implementing with log-transformation

Now implement the Viterbi algorithm with log-transformation. The steps are the same as above.

### Computation of the (log-transformed) $\omega$ table

In [60]:
def compute_w_log(model, x):
    k = len(model.init_probs)
    n = len(x)
    x = translate_observations_to_indices(x)
    
    w = make_table(k, n)
    
    # Base case: fill out w[i][0] for i = 0..k-1
    for j in range(k):
        ip = model.init_probs[j]
        ep = model.emission_probs[j][x[0]]
        w[j][0] = log(ip) + log(ep)
    
    # Inductive case: fill out w[i][j] for i = 0..k, j = 0..n-1
    for i in range(1, n):
        for j in range(k):
            max_prob = float('-inf')
            for l in range(k):
                tp = model.trans_probs[l][j]
                ep = model.emission_probs[j][x[i]]
                prob = w[l][i-1] + log(tp) + log(ep)
                if prob > max_prob:
                    max_prob = prob
            w[j][i] = max_prob
    return w  

In [61]:
import math  # Just ignore this :-)

def log(x):
    if x == 0:
        return float('-inf')
    return math.log(x)

### Finding the (log-transformed) joint probability of an optimal path

In [62]:
def opt_path_prob_log(w):
    max_prob = float('-inf')
    
    for j in range(len(w)):
        prob = w[j][-1]
        if prob > max_prob:
            max_prob = prob
    return max_prob

In [63]:
import numpy as np
w = compute_w_log(hmm_7_state, x_short)
np.array(w)
#opt_path_prob_log(w)

array([[        -inf,         -inf,         -inf,  -8.58173171,
         -9.85024304, -11.18774723, -13.0203287 , -14.10651847,
        -16.18596001, -17.54853784, -18.41158406, -21.18417278,
        -21.67128188, -23.12211476, -24.11446662, -26.38181258,
        -26.73403318, -28.04223798, -31.48538587, -31.44456388,
        -32.34730358, -36.12466668, -36.26045509, -37.28097783,
        -39.03101539, -40.74784224, -40.96188911, -43.4414415 ,
        -45.2352294 , -44.41965685, -48.95047989, -49.54029499,
        -49.19472607, -52.91907325, -54.76165131, -52.62432293,
        -57.4064604 , -59.28986046, -56.74706696, -61.711526  ,
        -63.30724398, -60.02251314, -66.93288232, -66.91916239,
        -63.9502845 , -71.8876592 , -71.91737517, -68.10119941,
        -74.78581114, -76.73326638],
       [        -inf,         -inf,  -6.9722938 ,  -8.46394868,
         -9.80145287, -11.63403434, -12.90254566, -14.79966565,
        -15.93909993, -17.0252897 , -19.57473487, -20.46730908,
   

Now do the same for `x_long`. What happens?

In [ ]:
w = compute_w_log(hmm_7_state, x_long)
opt_path_prob_log(w)

-1406.7209253880144

### Obtaining an optimal path through backtracking

In [ ]:
def backtrack_log(model, x, w):
    x = translate_observations_to_indices(x)
    
    max_prob = float('-inf')
    max_prob_position = None 
    for j in range(len(w)):
        prob = w[j][-1]
        if prob > max_prob:
            max_prob = prob
            max_prob_position = j
    path = str(max_prob_position)

    pre_prob = max_prob
    pre_prob_position = max_prob_position
    
    for i in range(-2, -len(x)-1, -1):
        for j in range(len(w)):
            tp = model.trans_probs[j][pre_prob_position]
            ep = model.emission_probs[pre_prob_position][x[i+1]]
            prob = w[j][i] + log(tp) + log(ep)
            if math.isclose(prob, pre_prob):
                pre_prob = w[j][i]
                pre_prob_position = j
                path = str(j) + path

    return path

In [ ]:
w = compute_w_log(hmm_7_state, x_short)
z_viterbi_log = backtrack_log(hmm_7_state, x_short, w)
z_viterbi_log

'3333333333321021021021021021021021021021021021021'

Now do the same for `x_long`. What happens?

In [ ]:
w = compute_w_log(hmm_7_state, x_long)
z_viterbi_log = backtrack_log(hmm_7_state, x_long, w)
z_viterbi_log

'333321021021021021021021021021021021021021021021021021021021021021021033333333334564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564563210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210321021021021021021021021033334564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564563333333456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456332102102102102102102102102102102102102102102102102102102102102102102102102102102102102102102102103210210210210210210210210210210210210210210210210210210210210210

### Does it work?

Think about how to verify that your implementations of Viterbi (i.e. `compute_w`, `opt_path_prob`, `backtrack`, and there log-transformed variants `compute_w_log`, `opt_path_prob_log`, `backtrack_log`) are correct.

One thing that should hold is that the probability of a most likely path as computed by `opt_path_prob` (or `opt_path_prob_log`) for a given sequence of observables (e.g. `x_short` or `x_long`) should be equal to the joint probability of a corersponding most probable path as found by `backtrack` (or `backtrack_log`) and the given sequence of observables. Why?

Make an experiment that validates that this is the case for your implementations of Viterbi and `x_short` and `x_long`.

In [ ]:
# To access joint_prob and joint_prob_log, you must copy your implementations from last week here ...

def joint_prob(model, x, z):
    x = translate_observations_to_indices(x)
    z = translate_path_to_indices(z)
    
    acc_prob = None
    
    if len(x) > 0:
        ip = model.init_probs[z[0]]
        ep = model.emission_probs[z[0]][x[0]]
        acc_prob = ip * ep
        
        for i in range(1, len(x)):
            tp = model.trans_probs[z[i-1]][z[i]]
            ep = model.emission_probs[z[i]][x[i]]
            acc_prob *= tp * ep
    
    return acc_prob

def joint_prob_log(model, x, z):
    x = translate_observations_to_indices(x)
    z = translate_path_to_indices(z)
    acc_prob = None
    
    if len(x) > 0:
        ip = model.init_probs[z[0]]
        ep = model.emission_probs[z[0]][x[0]]
        acc_prob = log(ip) + log(ep)
        
        for i in range(1, len(x)):
            tp = model.trans_probs[z[i-1]][z[i]]
            ep = model.emission_probs[z[i]][x[i]]
            acc_prob += log(tp) + log(ep)
    
    return acc_prob


# Check that opt_path_prob is equal to joint_prob(hmm_7_state, x_short, z_viterbi)

w = compute_w(hmm_7_state, x_short)
z_viterbi = backtrack(hmm_7_state, x_short, w)
print(opt_path_prob(w))
print(joint_prob(hmm_7_state, x_short, z_viterbi))

# Check that opt_path_prob_log is equal to joint_prob_log(hmm_7_state, x_short, z_viterbi_log)
w = compute_w_log(hmm_7_state, x_short)
z_viterbi_log = backtrack_log(hmm_7_state, x_short, w)

print(opt_path_prob_log(w))
print(joint_prob_log(hmm_7_state, x_short, z_viterbi_log))

# Do the above checks for x_long ...

w = compute_w_log(hmm_7_state, x_long)
z_viterbi_log = backtrack_log(hmm_7_state, x_long, w)

print(opt_path_prob_log(w))
print(joint_prob_log(hmm_7_state, x_long, z_viterbi_log))

w = compute_w(hmm_7_state, x_long)
z_viterbi = backtrack(hmm_7_state, x_long, w)

print(opt_path_prob(w))
print(joint_prob(hmm_7_state, x_long, z_viterbi))

1.9114255184318858e-31
1.9114255184318882e-31
-70.73228857440488
-70.73228857440486
-1406.7209253880144
-1406.7209253880178


TypeError: list indices must be integers or slices, not NoneType

Do your implementations pass the above checks?

### Does log-transformation matter?

Make an experiment that investigates how long the input string can be before `backtrack` and `backtrack_log` start to disagree on a most likely path and its probability.

In [ ]:
for i in range(1, len(x_long), 1):
    x = x_long[:i]
    z = z_long[:i]
    
    if i % 100 == 0 or i > 520:
        print(i)
        w = compute_w(hmm_7_state, x)
        z_viterbi = backtrack(hmm_7_state, x, w)
        w_log = compute_w_log(hmm_7_state, x)
        z_viterbi_log = backtrack_log(hmm_7_state, x, w_log)
        print(z_viterbi_log == z_viterbi)

100
True
200
True
300
True
400
True
500
True
521
True
522
True
523
True
524
True
525
True
526
True
527
True
528
True
529
True
530


TypeError: list indices must be integers or slices, not NoneType

**Your answer here:**

For the 7-state model, `backtrack` and `backtrack_log` start to disagree on a most likely path and its probability for **i = 530** like before.